In [6]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error 

In [21]:
# Load data from Excel file
data = pd.read_excel("C:\\Users\\msnin\\OneDrive\\\Desktop\\SPAC Research\\Dataset 6-2023.xlsx", sheet_name="OG (2)")
data.dropna(inplace=True)
data.shape 

features = data.iloc[:, 2:-1]  # assuming the target column is the last one
target = data['target']

X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=42)

linear regression with variables "ln_assets, ln_cash, ln_trust, warrant_liab, leverage_w, complexity, deSPAC, neg_fv_changes, ln_temp_equity, ln_temp_equity2, target" (no _cons variable)


In [22]:
model = LinearRegression()
model.fit(X_train, y_train)
predictions = model.predict(X_test)

mse = mean_squared_error(y_test, predictions)
print("Mean Squared Error:", mse)

# ADJUSTED RMSE (sqrt MSE with p and n)
p = X_train.shape[1] # number of predictors
n = len(y_test) # number of observations 
adjusted_rmse = np.sqrt(mse * (n - 1) / (n - p))
print("Adjusted RMSE:", adjusted_rmse)  



Mean Squared Error: 0.6703863873045869
Adjusted RMSE: 0.827375054007625


polynomial line of best fit (same variables, quadratic had the best RMSE -- tested degrees 0-10)

In [20]:
from sklearn.preprocessing import PolynomialFeatures

adj_rmse_dict = {}

for i in range(10):
        poly = PolynomialFeatures(i)
        X_poly_train = poly.fit_transform(X_train)
        X_poly_test = poly.transform(X_test)

        model = LinearRegression()
        model.fit(X_poly_train, y_train)

        y_pred = model.predict(X_poly_test)

        # Evaluate the model
        mse = mean_squared_error(y_test, y_pred)

        # ADJUSTED RMSE (sqrt MSE with p and n)
        p = X_train.shape[1] # number of predictors
        n = len(y_test) # number of observations 
        adjusted_rmse = np.sqrt(mse * (n - 1) / (n - p))
        adj_rmse_dict[i] = adjusted_rmse

print(adj_rmse_dict)



{0: 1.3953061904980122, 1: 0.827375054007625, 2: 0.7390372059675095, 3: 0.8396946998955493, 4: 6.537340275536831, 5: 1123.205095439545, 6: 60137197.20582778, 7: 191626017.4815785, 8: 714283179.1848011, 9: 854018836.7617644}


Ridge model (better at making long-term predictions, all variables are useful)

In [52]:
from sklearn.linear_model import Ridge, RidgeCV
from sklearn.preprocessing import PolynomialFeatures

poly = PolynomialFeatures(2)
X_poly_train = poly.fit_transform(X_train)
X_poly_test = poly.transform(X_test)

# # tuning alpha
# alphas = 10**np.linspace(10,-2,100)*0.5
# ridgecv = RidgeCV(alphas = alphas)
# ridgecv.fit(X_train, y_train)
# print(ridgecv.alpha_)

# best alpha is lowkey 0.5?
alpha = 0.5
model = Ridge(alpha=alpha)
model.fit(X_poly_train, y_train)

y_pred = model.predict(X_poly_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)

# ADJUSTED RMSE (sqrt MSE with p and n)
p = X_train.shape[1] # number of predictors
n = len(y_test) # number of observations 
adjusted_rmse = np.sqrt(mse * (n - 1) / (n - p))
print(adjusted_rmse)

0.7423129301934638
